In [1]:

import xarray as xr
from thesis_toolbox.utils import get_locations_CLP
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import yaml
locs = get_locations_CLP()

In [2]:
locs_df = get_locations_CLP().drop('YINCHUAN')
locs = ['SHAPOTOU','SACOL','BAODE','LUOCHUAN','LINGTAI','LANTIAN']

In [3]:
df = pd.read_csv('../../data/data/results/timeseries_table.csv', index_col=0)

In [4]:
df_wetdep2m = pd.DataFrame(columns=locs,
                      index=['North west CLP', 'Qaidam', 'Mongolia','Taklamakan','Gurbantunggut Desert','Central Asia'])
df_wetdep20m = df_wetdep2m.copy()
df_drydep2m = df_wetdep2m.copy()

df_drydep20m = df_drydep2m.copy()


In [5]:
df_m = df.mean(axis=0)

In [6]:
def get_depval_from_source(df_out,df_in,locs,source, kind, psize, convert_fact=None):
    trans_dict = {
        'Qaidam':"quaidam_basin",
        'Central Asia':'central_asia',
        "Gurbantunggut Desert":"jungger_basin",
        "Taklamakan":"taklamakan",
        "North west CLP": "north_west",
        "Mongolia": "mongolia"
        
    }
    for site in locs:
        if convert_fact:
            df_out.loc[source][site] = df_in.loc[f'{trans_dict[source]} {site} {kind} {psize}']*convert_fact
        else:
            df_out.loc[source][site] = df_in.loc[f'{trans_dict[source]} {site} {kind} {psize}']
    return df_out

In [7]:
conv_fact = 1e3/10800

for source in df_wetdep2m.index:
    df_wetdep2m = get_depval_from_source(df_wetdep2m, df_m, locs,source,'wetdep','2micron',conv_fact)
    df_wetdep20m = get_depval_from_source(df_wetdep20m, df_m, locs,source,'wetdep','20micron',conv_fact)
    df_drydep2m = get_depval_from_source(df_drydep2m, df_m, locs,source,'drydep','2micron',conv_fact)
    df_drydep20m = get_depval_from_source(df_drydep20m, df_m, locs,source,'drydep','20micron',conv_fact)

In [8]:
df_totdep2m = (df_drydep2m + df_wetdep2m).copy()
df_totdep20m = (df_drydep20m + df_wetdep20m).copy()

df_totdep2m = df_totdep2m.astype(np.float64).round(3)
df_totdep20m = df_totdep20m.astype(np.float64).round(3)

In [9]:
df_multi20 = pd.DataFrame(columns=locs,
    index=[
    ['North west CLP','North west CLP', 'Qaidam','Qaidam', 'Mongolia','Mongolia','Taklamakan','Taklamakan','Gurbantunggut Desert','Gurbantunggut Desert','Central Asia','Central Asia'],
    ['Wet','Tot','Wet','Tot','Wet','Tot','Wet','Tot','Wet','Tot','Wet','Tot']
                            ])

df_multi2 = df_multi20.copy()

In [10]:
for s in df_multi20.index:
    if s[1] == 'Wet':
        df_multi20.loc[s] = df_wetdep20m.loc[s[0]].astype(np.float64).round(3)
    else:
        df_multi20.loc[s] = df_totdep20m.loc[s[0]]

In [11]:
for s in df_multi2.index:
    if s[1] == 'Wet':
        df_multi2.loc[s] = df_wetdep2m.loc[s[0]].astype(np.float64).round(3)
    else:
        df_multi2.loc[s] = df_totdep2m.loc[s[0]]

In [12]:
# Store tables and export to latex 
with open('tab20m_dep.txt','w') as f:
    df_totdep20m.to_latex(f)
with open('tabmulti20.txt', 'w') as f:
    df_multi20.to_latex(f)
with open('tabmulti2.txt', 'w') as f:
    df_multi2.to_latex(f)

/tmp/ipykernel_588/1112820374.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_totdep20m.to_latex(f)
/tmp/ipykernel_588/1112820374.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_multi20.to_latex(f)
/tmp/ipykernel_588/1112820374.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to us